# EnergyPlus Parameterization and Sensitivity Analysis for Residential Single Family Homes


## Daniel Xu, Keller Lab

This notebook was created in order to document the workflow and processes of EnergyPlus within Python. This is for my project involving the parameterization of EnergyPlus. Given that EnergyPlus has over 2000 available inputs, more research needs to be done in order to document which parameters are most significant and relevant in home energy modeling and simulations. 

The goal of this project is to run EnergyPlus on a model single family residential home. Some of the inputs will be fixed to default variables. Other variables, such as window thickness, thermostat set-points, etc. will be varied on pre-defined distributions. 

The distributions will be defined as seen in this paper: 
https://www.sciencedirect.com/science/article/abs/pii/S037877881631372X?via=ihub

### Importing necessary libraries

In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

### Changing the working directory 

In [2]:
os.chdir("/jumbo/keller-lab/projects/eplus_sensitivity/scripts")
print("Current working directory:", os.getcwd())

Current working directory: /jumbo/keller-lab/projects/eplus_sensitivity/scripts


### Setting EnergyPlus Input Data Dictionary

In [3]:
iddfile = "/jumbo/keller-lab/Applications/EnergyPlus-24-1-0/Energy+.idd"
IDF.setiddname(iddfile)

### Setting a Skeleton IDF File 

This modifiable IDF file is provided by EnergyPlus in its initial download. It provides a mock residential home in Chicago with multiple zones. I will assume that some of the inputs are fixed and others are varied on a distribution. 

Directly from the EnergyPlus documentation: 

"This file does the basic test of an air distribution system in a residential home. A two speed heat pump with a supplmental gas heater provides space heating and cooling. It provides ventilation through the ZoneAirBalance:OutdoorAir model."

In [4]:
idfname = "/jumbo/keller-lab/Applications/ExampleFiles/SingleFamilyHouse_TwoSpeed_MultiStageElectricSuppCoil.idf"


epwfile = 